# Stabilité des algorithmes

In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy import interpolate

In [ ]:
## Newton formulation 
def compute_divided_diff_coef(x, y):
    coef = y.copy()
    for i in range(1,x.size):
        coef[i:] = (coef[i:]-coef[i-1:-1])/(x[i:] - x[:-i])
    return coef

def poly_newton_interp(coef, xi, x):
    n = coef.size
    p = np.zeros(x.size)
    for i in range(n-1,0,-1):
        p = (coef[i]+p) * (x-xi[i-1])
    p = p + coef[0] 
    return p

def newton_interp(xi, yi):
    coef = compute_divided_diff_coef(xi, yi)
    return lambda x: poly_newton_interp(coef, xi, x)

## Lagrange formulation
def lagrange(k, xi, x):
    lag = np.ones(x.size)
    for i in range(xi.size):
        if (i!=k): lag *= (x-xi[i])/(xi[k]-xi[i])
    return lag

def poly_lagrange_interp(xi, yi, x):
    p = np.zeros(x.size)
    for i in range(xi.size):
        p += lagrange(i, xi, x) * yi[i]
    return p
        
def lagrange_interp(xi, yi):
    return lambda x: poly_lagrange_interp(xi, yi, x)


# Formulation barycentrique
def compute_bary_coef(xk):
    n = xk.size
    wk = 0. * np.copy(xk)

    for k in range(n):
        wk[k] = 1. / (np.prod( xk[k] - xk[np.arange(n)!=k] ))
        
    return wk

def poly_barycentric_interp(xk, yk, x):
    n = xk.size
    l = 0. * np.copy(x) + 1.
    p = 0. * np.copy(x)
    
    wk = compute_bary_coef(xk)
    
    mask = np.in1d(x, xk[:])
    p[np.where(mask)] = yk[np.where(np.in1d(xk, x))]
    mask = np.invert(mask)    

    for k in range(n):
        l[mask] *= x[mask] - xk[k]
        p[mask] += (wk[k]/(x[mask] - xk[k])) * yk[k]
        
    return l*p

def barycentric_interp(xi, yi):
    return lambda x: poly_barycentric_interp(xi, yi, x)

## Cas du polycopié avec un ensemble de points quelconques

**Polynôme d'interpolation d'ordre 5**

In [ ]:
n = 5
xi = np.array([ 0., 2., 4., 5., 8., 10.])
yi = np.array([-1., 1., 6., 0., 2.,  5.])

p = newton_interp(xi, yi)
ps = interpolate.KroghInterpolator(xi, yi)
plag = lagrange_interp(xi, yi)
pslag = interpolate.lagrange(xi, yi)
pbar = barycentric_interp(xi, yi)


xmin = -1; xmax = 11.
x = np.linspace(xmin, xmax, 100)

fig = go.Figure(layout_yaxis_range=[-10,15])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x) (newton)"))
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x) (newton scipy)", line_dash="dot"))
fig.add_trace(go.Scatter(x=x, y=plag(x), name="p(x) (lagrange)"))
fig.add_trace(go.Scatter(x=x, y=plag(x), name="p(x) (lagrange scipy)", line_dash="dot"))
fig.add_trace(go.Scatter(x=x, y=pbar(x), name="p(x) (barycentric)"))

fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()

## Exemple pour la fonction $f(x) = sin(x)$

In [ ]:
def f(x):
    return np.sin(x)

### Formule de Lagrange

In [ ]:
n = 25

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

p = lagrange_interp(xi, yi)
ps = interpolate.lagrange(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
fig.add_trace(go.Scatter(x=x, y=ps(x), name="p(x) scipy"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()

### Formule de Newton

In [ ]:
n = 60

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

p = newton_interp(xi, yi)
ps = interpolate.KroghInterpolator(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
fig.add_trace(go.Scatter(x=x, y=ps(x), name="p(x) scipy"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()

### Formule d'interpolation barycentrique

In [ ]:
n = 60

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

p = barycentric_interp(xi, yi)
ps = interpolate.BarycentricInterpolator(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
fig.add_trace(go.Scatter(x=x, y=ps(x), name="p(x) scipy"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()